In [ ]:
import os
import yaml
import pandas as pd
import matplotlib as plt

from matplotlib import rcParams
from pyathena import connect
from pyathena.util import as_pandas
from sqlalchemy import *

%matplotlib inline
%load_ext autoreload

In [ ]:
pg_cred = yaml.load(open("../conf/local/credentials.yml"), Loader=yaml.FullLoader)


In [ ]:
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(pg_cred["pg_user"], pg_cred["pg_pass"], pg_cred["pg_host"], 5432, "iefp")

# The return value of create_engine() is our connection object
con = create_engine(url, client_encoding='utf8')

# We then bind the connection to MetaData()
meta = MetaData(bind=con, reflect=True)

## SQL queries to extract intervention information

In [ ]:
# Get intervention information all togetehr in one dataframe. 
# Remove duplicate rows by only looking at results (tipo_movimento >30).
# Remove 'intervention codes' which are not actual interventions (PPE).

sql = """select
ute_id,idade,sexo,ppe,ano_mes,data_intervencao,intervencoes.tipo_movimento,centro,codigo_intervencao,
resultado_intervencao, dcodigo_interv 

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

where ((intervencoes.tipo_movimento > 30) 

and intervencoes.codigo_intervencao != '0101' 
and intervencoes.codigo_intervencao != '0102')

order by ute_id

;"""

interv_descript = pd.read_sql(sql, con)


In [ ]:
# Count the number of different types of interventions for output as a csv

sql = """select

codigo_intervencao, dcodigo_interv, count(codigo_intervencao) as count_code

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

where (intervencoes.tipo_movimento > 30) 

group by codigo_intervencao, dcodigo_interv 

;"""

count_int = pd.read_sql(sql, con)


## Write counts to csv for partners to view

In [ ]:
# Sort by count
count_int_sort = count_int.sort_values(by= 'count_code', ascending=False, inplace=False)

In [ ]:
count_int_sort['codigo_intervencao'] = count_int_sort['codigo_intervencao'].astype('int64')

In [ ]:
# Get total count
total = count_int_sort.count_code.sum().astype('int64')

In [ ]:
total

In [ ]:
# Make percentage column
count_int_sort['perc'] = (count_int_sort.count_code/ total) * 100

In [ ]:
# Write to csv for partners to view
count_int_sort.to_csv('count_int.csv',sep=',', header=True) 

## Intervention descriptions and types

In [ ]:
interv_descript.head(5)

In [ ]:
interv_descipt_nodup = interv_descript.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
interv_descipt_nodup.head(5)

In [ ]:
interv_descript['dcodigo_interv'].unique()

## Average number of interventions per person

In [ ]:
interv_descipt_nodup.head(5)

In [ ]:
counts=interv_descipt_nodup.groupby(['ute_id']).size().reset_index(name='counts')

print(counts.mean()) # 6 interventions avg (not including PPE)
print(counts.min()) # 1
print(counts.max()) # 59
counts.counts.value_counts()

In [ ]:
# How many interventions have people taken

g=sns.distplot(counts['counts'], bins=29) #zoom
g.set_xlabel('Number of interventions')
g.set_ylabel('Proportion of job seekers')

In [ ]:
counts['counts'].value_counts() # most people have 2 interventions

## Distribution of interventions

In [ ]:
interv_descript_nodup['codigo_intervencao'].value_counts()

In [ ]:
code_counts = interv_descript_nodup['codigo_intervencao'].value_counts()

In [ ]:
interv_descript_nodup.head()

In [ ]:
# Description
count_all = interv_descript_nodup.dcodigo_interv.value_counts()
count_all_df = count_all.rename_axis('codes').reset_index(name='counts')
count_all_df.codes = count_all_df.codes.astype('category')
count_all_df.head(10)

In [ ]:
# Codes
count_all = interv_descript_nodup.codigo_intervencao.value_counts()
count_all_df = count_all.rename_axis('codes').reset_index(name='counts')
count_all_df.codes = count_all_df.codes.astype('category')
count_all_df.head(10)

0 INTERVIEW PLACEMENT 1372
1 CONTINUOUS TRAINING - MODULAR TRAINING 916
2 ACTIVE JOB SEARCH (E.G.) 728
3 FORMATION - ACTIVE LIFE 691
4 COLLECTIVE INFORMATION SESSION 297
5 SOCIAL INSERTION INCOME - RSI 294
6 EMPLOYMENT-INSERTION CONTRACT 265
7 ORIENTATION INTERVIEW 184
8 SOCIAL INSERTION INCOME - END 170
9 PLACING 165

In [ ]:
count_all_df_s = count_all_df[0:25]
count_all_df_s

In [ ]:
import seaborn as sns
#plt.figure(figsize=(15, 15))
import matplotlib.pylab as plt

hist = sns.barplot(count_all_df_s['codes'].astype('category'), count_all_df_s['counts'])
hist.set_ylabel('Intervention Code Count')
hist.set_xticklabels(hist.get_xticklabels(), rotation=30)

In [ ]:
interv_descipt_nodup['codigo_intervencao'].astype('category') 

In [ ]:
code_counts.head() # interview placement = 7003

In [ ]:
interv_descipt_nodup.head()
#interv_descipt_nodup.sexo.value_counts()

In [ ]:
# by sex
counts=interv_descipt_nodup[['ute_id','sexo']].groupby(['ute_id','sexo']).size().reset_index(name='counts')
#counts=interv_descipt_nodup.groupby(['sexo']).size().reset_index(name='counts')
counts.head()


In [ ]:
g=sns.barplot(counts['sexo'],counts['counts']) 
g.set_xlabel('Sex', fontsize=18)
g.set_ylabel('Average number of interventions', fontsize=18)

In [ ]:
# by age 
counts=interv_descipt_nodup[['ute_id','idade']].groupby(['ute_id','idade']).size().reset_index(name='counts')
#counts=interv_descipt_nodup.groupby(['sexo']).size().reset_index(name='counts')
counts.head()

In [ ]:
# figure size in inches
rcParams['figure.figsize'] = 10,7

In [ ]:
counts.idade.dtype

In [ ]:
counts.idade = pd.to_numeric(counts.idade, errors='coerce')

In [ ]:
g=sns.regplot(counts.idade,counts['counts']) 
g.set_xlabel('Age (years)', fontsize=18)
g.set_ylabel('Average number of interventions', fontsize=18)

## Outcome codes 

In [ ]:
# Count results per person

counts=interv_descipt_nodup[['ute_id','sexo','resultado_intervencao']].groupby(['ute_id','sexo','resultado_intervencao']).size().reset_index(name='counts')
#counts=interv_descipt_nodup.groupby(['sexo']).size().reset_index(name='counts')
counts.head()

In [ ]:
counts.resultado_intervencao.value_counts()

In [ ]:
g=sns.barplot(counts['resultado_intervencao'].astype('category'),counts['counts']) 
g.set_xlabel('Intervention results', fontsize=18)
g.set_ylabel('Frequency of result type', fontsize=18)
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
p=counts['resultado_intervencao'].astype('category').value_counts().plot(kind="bar")
p.set(xlabel="Intevention Result Codes", ylabel="Frequency")
plt.rcParams.update({'font.size': 18})

In [ ]:
# Get results of interventions
sql = """select
ute_id,idade,sexo,ppe,ano_mes,data_intervencao,intervencoes.tipo_movimento,centro,codigo_intervencao,
resultado_intervencao, dcodigo_interv, dresultado 

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

join resultado_intervencoes

on resultado_intervencoes.cresultado ilike intervencoes.resultado_intervencao

where ((intervencoes.tipo_movimento > 30) 

and intervencoes.codigo_intervencao != '0101' 
and intervencoes.codigo_intervencao != '0102')

order by ute_id

limit 10000
;"""

interv_descript_res = pd.read_sql(sql, con)

In [ ]:
interv_descript_res.head(20)